# Application finale : Prédictions Vrais/ Faux billets

In [1]:
import pandas as pd
import numpy as np
import pickle
import ipywidgets as widgets
from IPython.display import display, FileLink, HTML, clear_output
import io
from sklearn.preprocessing import StandardScaler

# Ignorer tous les avertissements
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Charger le modèle de régression linéaire à partir du fichier pickle
with open('modele_RegLog.pickle', 'rb') as file:
    modele_RegLog = pickle.load(file)
    
# Charger le modèle du KMeans à partir du fichier pickle
with open('modele_Kmeans.pickle', 'rb') as file:
    modele_Kmeans = pickle.load(file)
    
# Charger 'sélection' à partir du fichier pickle
with open('selection.pickle', 'rb') as file:
    selection = pickle.load(file)
    
# Charger le dictionnaire d'identification des clusters KMeans à partir du fichier pickle
with open('id_Kmeans.pickle', 'rb') as file:
    clusters = pickle.load(file)

In [3]:
# Créer les widgets FileUpload et Dropdown
upload = widgets.FileUpload(accept='.csv')
choix = widgets.Dropdown(options=['Modèle de régression logistique', 'Modèle du KMeans'], 
                         value='Modèle de régression logistique', description='Modèle : ', disabled=False)

# Créer un conteneur pour afficher le DataFrame
output = widgets.Output()

def fichier_upload(change):
    # Vérifier si le fichier a été chargé
    if len(upload.value) > 0:
        # Lire les informations du fichier 
        file_info = upload.value[0]
        # Lire le Dataframe et mettre les 'id' des billets en index
        df = pd.read_csv(io.BytesIO(file_info['content']), index_col='id')
        # Afficher le DataFrame dans le conteneur
        with output:
            if choix.value == 'Modèle de régression logistique':
                # X matrice des données avec les variables sélectionnées
                selected_column_indices = [df.columns.get_loc(col) for col in df.columns[selection]]
                X = df.iloc[:, selected_column_indices]
                # Prédictions avec le modèle entraîné choisi
                predictions = modele_RegLog.predict(X)
                # Rajout de la colonne 'Prédictions'
                df['Prediction'] = np.where(predictions == 1, 'Vrai billet', 'Faux billet')
                # Calcul des probabilités
                probabilités = modele_RegLog.predict_proba(X)
                # Rajout de la colonne d'indice de confiance
                df['Indice de confiance'] = np.max(probabilités, axis=1)
                # Afficher le dataframe avec les prédictions
                display(df)
                # Créer le bouton "Download" seulement lorsque le DataFrame est affiché
                download_button = widgets.Button(description="Download")
                # Lier la fonction fichier_download au clic sur le bouton "Download"
                download_button.on_click(lambda x: fichier_download(df))
                display(download_button)
            elif choix.value == 'Modèle du KMeans':
                # X matrice des données
                X = df.values
                # Il faut scaller les données car le modèle a été entraîné avec des données centrées et réduites
                scaler = StandardScaler()
                scaler.fit(X)
                X_scaled = scaler.transform(X)
                # Prédictions avec le modèle entraîné choisi
                predictions = modele_Kmeans.predict(X_scaled)
                # Rajout de la colonne 'Prédictions'
                df['Prediction'] = predictions
                # Identification des clusters
                df['Prediction'] = df['Prediction'].map(clusters)
                # Changer le type de donnée par l'affichage souhaité 
                df['Prediction'] = np.where(df['Prediction'] == 1, 'Vrai billet', 'Faux billet')
                # Afficher le dataframe avec les prédictions
                display(df)
                # Créer le bouton "Download" seulement lorsque le DataFrame est affiché
                download_button = widgets.Button(description="Download")
                # Lier la fonction fichier_download au clic sur le bouton "Download"
                download_button.on_click(lambda x: fichier_download(df))
                display(download_button)

# Créer une fonction pour afficher le bouton DL
def fichier_download(df):
    if choix.value == 'Modèle de régression logistique':
        # Sauvegarder le DataFrame avec les prédictions et l'indice de confiance dans un fichier CSV
        df.to_csv('billets_predictions_Regression_Logistique.csv', sep = '\t', index=True)
        # Créer le lien de téléchargement
        download_link = FileLink('billets_predictions_Regression_Logistique.csv', 
                                 result_html_prefix="Cliquez ici pour télécharger : ")
        display(download_link)
    elif choix.value == 'Modèle du KMeans':
        # Sauvegarder le DataFrame avec les prédictions et l'indice de confiance dans un fichier CSV
        df.to_csv('billets_predictions_KMeans.csv', sep = '\t', index=True)
        # Créer le lien de téléchargement
        download_link = FileLink('billets_predictions_KMeans.csv', 
                                 result_html_prefix="Cliquez ici pour télécharger : ")
        display(download_link)
        
# Créer une fonction de réinitialisation
def reinitialiser_widget(btn):
    # Effacer le fichier chargé précédemment
    upload.value = ()
    # Effacer le DataFrame affiché précédemment
    output.clear_output()

# Lier la fonction fichier_upload au chargement du fichier
upload.observe(fichier_upload, names='value')

# Lier la fonction de réinitialisation à l'événement de changement du dropdown
choix.observe(reinitialiser_widget, names='value')

# Afficher le widget 'choix'
display(choix)

# Ajouter un commentaire devant le bouton "Upload"
upload_comment = HTML('<p>Choisissez un fichier CSV à charger :</p>')
display(upload_comment)

# Afficher le widget 'upload'
display(upload)

# Afficher le conteneur pour le DataFrame
display(output)

Dropdown(description='Modèle : ', options=('Modèle de régression logistique', 'Modèle du KMeans'), value='Modè…

FileUpload(value=(), accept='.csv', description='Upload')

Output()